In [1]:
import pandas as pd
from datetime import datetime, timezone, timedelta
import numpy as np
import time
import os
from dotenv import load_dotenv

# get data within specific date 

In [2]:
data_5_11_des = donasi_download[(donasi_download['Date'] >= "2025-12-19") & (donasi_download['Date'] <= "2025-12-26")]
data_5_11_des.to_excel("data_19-25_desember_teman_baik.xlsx")

NameError: name 'donasi_download' is not defined

# test

In [2]:
import pandas as pd

def pandas_upsert_with_stats(
    df_main: pd.DataFrame,
    df_update: pd.DataFrame,
    key,
    allow_schema_evolution: bool = True,
    verbose: bool = True
) -> pd.DataFrame:
    """
    Perform a selective UPSERT from df_update into df_main with statistics.

    Prints:
    - Rows updated
    - Rows inserted
    - Columns added
    - Per-column change counts
    """

    main = df_main.copy()
    update = df_update.copy()

    # -------------------------------
    # Schema evolution
    # -------------------------------
    added_columns = []
    if allow_schema_evolution:
        added_columns = list(set(update.columns) - set(main.columns))
        for col in added_columns:
            main[col] = pd.NA

    # -------------------------------
    # Validation
    # -------------------------------
    if key not in main.columns or key not in update.columns:
        raise ValueError(f"Key '{key}' must exist in both dataframes")

    # -------------------------------
    # Index alignment
    # -------------------------------
    main = main.set_index(key)
    update = update.set_index(key)

    common_cols = list(set(main.columns) & set(update.columns))

    # -------------------------------
    # Detect updates BEFORE applying
    # -------------------------------
    intersect_idx = main.index.intersection(update.index)

    change_counts = {}
    rows_with_any_change = set()

    for col in common_cols:
        old = main.loc[intersect_idx, col]
        new = update.loc[intersect_idx, col]

        mask = new.notna() & (old != new)
        change_counts[col] = int(mask.sum())

        rows_with_any_change.update(intersect_idx[mask])

    rows_updated = len(rows_with_any_change)

    # -------------------------------
    # Apply update (NULL-safe)
    # -------------------------------
    main.update(update[common_cols])

    # -------------------------------
    # Insert new rows
    # -------------------------------
    new_rows = update.loc[~update.index.isin(main.index)]
    rows_inserted = len(new_rows)

    main = pd.concat([main, new_rows], axis=0)

    result = main.reset_index()

    # -------------------------------
    # Logging
    # -------------------------------
    if verbose:
        print("UPSERT SUMMARY")
        print("-" * 40)
        print(f"Rows updated        : {rows_updated}")
        print(f"Rows inserted       : {rows_inserted}")
        print(f"Columns added       : {len(added_columns)}")

        if added_columns:
            print(f"  Added columns     : {added_columns}")

        changed_cols = {k: v for k, v in change_counts.items() if v > 0}
        if changed_cols:
            print("Column changes:")
            for col, cnt in changed_cols.items():
                print(f"  {col:<15} : {cnt}")
        else:
            print("Column changes      : none")

        print("-" * 40)

    return result


# Load Data

In [3]:
pd.set_option('display.max_columns', None)

In [10]:
data_local = pd.read_excel("../../analisa/2025/Summary_2025.xlsx")

In [11]:
data_local['Source'] = "Web Ads"
data_local.to_excel("../../analisa/2025/Summary_2025.xlsx")

Load data from sql

In [12]:
not_the_time = timezone(timedelta(hours=7))
now = datetime.now(not_the_time).hour

if now < 13:
    print("❌ 13:00 yet, which means you cant do a query to database\n",)
    raise SystemExit("Stopping cell due to not the right time.")

In [13]:
from sqlalchemy import create_engine
from urllib.parse import quote_plus

load_dotenv()

username_db = os.getenv("DB_USERNAME")
password_db = os.getenv("DB_PASSWORD")
host_db = os.getenv("DB_HOST")
port_db = os.getenv("DB_PORT")
database_db = os.getenv("DB_DATABASE")

# Database credentials

# URL-encode username & password in case they have special characters
username_enc = quote_plus(username_db)
password_enc = quote_plus(password_db)

# Create SQLAlchemy engine
engine = create_engine(
    f"mysql+pymysql://{username_enc}:{password_enc}@{host_db}:{port_db}/{database_db}"
)

# Run a query and load results into Pandas
query_donasi = """SELECT a.*,b.title
FROM wp_ajskr.tEZ3UbOt_dja_donate a
LEFT JOIN wp_ajskr.tEZ3UbOt_dja_campaign b ON b.campaign_id = a.campaign_id """
query_aff_submit = """SELECT *
FROM wp_ajskr.tEZ3UbOt_dja_aff_submit"""
query_aff_code = """SELECT *
FROM wp_ajskr.tEZ3UbOt_dja_aff_code"""
query_users = """SELECT *
FROM wp_ajskr.tEZ3UbOt_users"""

donasi_table = pd.read_sql(query_donasi, engine)
aff_submit_table = pd.read_sql(query_aff_submit, engine)
aff_code_table = pd.read_sql(query_aff_code, engine)
user_table = pd.read_sql(query_users, engine)

In [14]:
# Join Table
donasi_submit = donasi_table.merge(aff_submit_table, left_on='id', right_on='donate_id', how='left')
donasi_submit_code = donasi_submit.merge(aff_code_table, left_on='affcode_id', right_on='id', how='left')
donasi_users = donasi_submit_code.merge(user_table, left_on='user_id_y', right_on='ID', how='left')


# Choose only necessary columns
donasi_download = donasi_users[['campaign_id','invoice_id', 'name', 'whatsapp', 'email', 'ip', 'comment', 'title','nominal', 'payment_method', 'payment_number', 'payment_account', 'status', 'nominal_commission', 'display_name', 'aff_code', 'created_at_x', 'info_qurban', 'info_package2', 'info_zfitrah', 'info_donate', 'utm_source', 'utm_medium', 'utm_content', 'utm_campaign', 'utm_term', 'utm_id']]

In [15]:
# Mapping campaign and campaigner
campaign_map = {
    "dja3d0s0b0k":"pungky",
    "djah4o94i2w":"cekat",
    "djazb6xk1xx": "fahry",
    "djayr4jyoxw": "fahry",
    "djax0gtszee": "alif",
    "djauvpug3dk": "fahry",
    "djar8al0qg2": "ali",
    "djam1ptsz9a": "dedi",
    "djait62m3qv": "fatih",
    "djah4o94i2w": "cekat",
    "djabkim8p3t": "fredo",
    "dja9o7ftxxf": "cekat"
}

donasi_download["zisco"] = donasi_download["campaign_id"].map(campaign_map)  # returns "alif" or None if not found
donasi_download['zisco'].value_counts(dropna=False)

C:\Users\faruq\AppData\Local\Temp\ipykernel_4420\1842361687.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  donasi_download["zisco"] = donasi_download["campaign_id"].map(campaign_map)  # returns "alif" or None if not found


zisco
NaN       36417
fahry     18871
fredo     17601
alif      12003
fatih     11965
ali         955
dedi        153
cekat        15
pungky        2
Name: count, dtype: int64

# Feature engineering data from SQL

In [16]:
donasi_download['Day'] = donasi_download['created_at_x'].dt.strftime("%a")
donasi_download['Time'] = donasi_download['created_at_x'].dt.strftime("%H:%M:%S")
donasi_download = donasi_download.rename(columns={
    "invoice_id": "Invoice ID",
    "name": "Donatur",
    "whatsapp": "Whatsapp",
    "email": "Email",
    "comment": "Comment",
    "title": "Program",
    "nominal":"Total",
    "payment_method": "Payment Method",
    "payment_number": "Payment Number",
    "payment_account": "Payment Account",
    "status": "Payment Status",
    "nominal_commission":"Fundraiser Commision",
    "display_name":"Fundraiser Name",
    "created_at_x":"Date",
    "info_qurban":"Data Qurban",
    "info_package2":"Data Package-2",
    "info_zfitrah":"Data Zakat Fitrah",
    "info_donate":"Additional Data",
    "utm_source":"UTM Source",
    "utm_medium":"UTM Medium",
    "utm_content":"UTM Content",
    "utm_campaign":"UTM Campaign",
    "utm_term":"UTM Term",
    "utm_id":"UTM ID"    
})

donasi_download["Payment Status"] = np.where(donasi_download['Payment Status'] == 1, "Success", "Waiting")

donasi_download['Source'] = "Web Ads"

C:\Users\faruq\AppData\Local\Temp\ipykernel_4420\228523208.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  donasi_download['Day'] = donasi_download['created_at_x'].dt.strftime("%a")
C:\Users\faruq\AppData\Local\Temp\ipykernel_4420\228523208.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  donasi_download['Time'] = donasi_download['created_at_x'].dt.strftime("%H:%M:%S")


In [17]:
# save data donation from database to local
donasi_download.to_excel("data_payment.xlsx")

In [18]:
# donasi_download = pd.read_excel("data_payment.xlsx")

data_from_web = pd.read_excel("checkpoint_files/data_from_web_updated.xlsx")

Load data sql that frequently updated by donasi download

In [19]:
data_from_web.drop_duplicates(subset=['Invoice ID'], inplace=True)
donasi_download.drop_duplicates(subset=['Invoice ID'], inplace=True)

In [20]:
# data_from_web_updated = update_data_web(
#     main_df=data_from_web,
#     updated_df=donasi_download,
#     key_col="Invoice ID",
#     cols_to_check=["Payment Status"]
# )

data_from_web_updated = pandas_upsert_with_stats(
    df_main=data_from_web,
    df_update=donasi_download,
    key="Invoice ID"
)

C:\Users\faruq\AppData\Local\Temp\ipykernel_4420\3212065580.py:68: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[81357157196.0 82157720711.0 8561889392.0 ... '087846860696' '08128241367'
 '087871662399']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  main.update(update[common_cols])
C:\Users\faruq\AppData\Local\Temp\ipykernel_4420\3212065580.py:68: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan nan nan ... '120241746817440202' '120240573983310706' '']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  main.update(update[common_cols])


UPSERT SUMMARY
----------------------------------------
Rows updated        : 96954
Rows inserted       : 1020
Columns added       : 1
  Added columns     : ['campaign_id']
Column changes:
  Email           : 60756
  UTM Term        : 3336
  UTM ID          : 96546
  Comment         : 47281
  campaign_id     : 95492
  Payment Status  : 113
  UTM Source      : 3084
  UTM Medium      : 3085
  UTM Content     : 3093
  Whatsapp        : 96954
  Payment Number  : 4
  Total           : 41
  Donatur         : 19
  UTM Campaign    : 3085
----------------------------------------


In [21]:
# df_final.reset_index(drop=True, inplace=True)

# 🔥 Also remove any "Unnamed" columns just in case
# df_final = df_final.loc[:, ~df_final.columns.str.contains("^Unnamed")]
data_from_web_updated = data_from_web_updated[['Invoice ID', 'Donatur', 'Whatsapp',
       'Email', 'ip', 'Comment', 'Program', 'Total', 'Payment Method',
       'Payment Number', 'Payment Account', 'Payment Status',
       'Fundraiser Commision','aff_code', 'Fundraiser Name','zisco', 'Date', 'Day', 'Time', 'Data Qurban',
       'Data Package-2', 'Data Zakat Fitrah', 'Additional Data', 'UTM Source',
       'UTM Medium', 'UTM Content', 'UTM Campaign', 'UTM Term', 'UTM ID','Source']]

data_from_web_updated.to_excel("checkpoint_files/data_from_web_updated.xlsx")

# Next Step (main)

In [22]:
data_local['Invoice ID'].duplicated().sum()

np.int64(0)

In [23]:
data_local_clean = data_local[~data_local["Invoice ID"].isin(data_from_web["Invoice ID"])]

In [24]:
data_all = pd.concat([data_local_clean, data_from_web_updated])
data_all

,Unnamed: 0.72,Unnamed: 0.71,Unnamed: 0.70,Unnamed: 0.69,Unnamed: 0.68,Unnamed: 0.67,Unnamed: 0.66,Unnamed: 0.65,Unnamed: 0.64,Unnamed: 0.63,Unnamed: 0.62,Unnamed: 0.61,Unnamed: 0.60,Unnamed: 0.59,Unnamed: 0.58,Unnamed: 0.57,Unnamed: 0.56,Unnamed: 0.55,Unnamed: 0.54,Unnamed: 0.53,Unnamed: 0.52,Unnamed: 0.51,Unnamed: 0.50,Unnamed: 0.49,Unnamed: 0.48,Unnamed: 0.47,Unnamed: 0.46,Unnamed: 0.45,Unnamed: 0.44,Unnamed: 0.43,Unnamed: 0.42,Unnamed: 0.41,Unnamed: 0.40,Unnamed: 0.39,Unnamed: 0.38,Unnamed: 0.37,Unnamed: 0.36,Unnamed: 0.35,Unnamed: 0.34,Unnamed: 0.33,Unnamed: 0.32,Unnamed: 0.31,Unnamed: 0.30,Unnamed: 0.29,Unnamed: 0.28,Unnamed: 0.27,Unnamed: 0.26,Unnamed: 0.25,Unnamed: 0.24,Unnamed: 0.23,Unnamed: 0.22,Unnamed: 0.21,Unnamed: 0.20,Unnamed: 0.19,Unnamed: 0.18,Unnamed: 0.17,Unnamed: 0.16,Unnamed: 0.15,Unnamed: 0.14,Unnamed: 0.13,Unnamed: 0.12,Unnamed: 0.11,Unnamed: 0.10,Unnamed: 0.9,Unnamed: 0.8,Unnamed: 0.7,Unnamed: 0.6,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,No,Invoice ID,Donatur,Sapaan,Nominal,Kode Unik,Total,Whatsapp,Email,Comment,Program,Payment Method,Payment Number,Payment Account,Payment Status,Fundraiser Commission,Fundraiser Name,Date,Day,Time,UTM Source,UTM Medium,Source,ip,Fundraiser Commision,aff_code,zisco,Data Qurban,Data Package-2,Data Zakat Fitrah,Additional Data,UTM Content,UTM Campaign,UTM Term,UTM ID
5,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,6.0,INV-2508040IVSI,Mayang,Bapak,100000.0,446.0,100446.0,82162426297.0,NaN,NaN,INFAK PALESTINA: BENTUK PEDULI KITA KEPADA SAU...,instant,https://m.dana.id/s/m7v7pwbn,DANA,Waiting,1004.0,Teman Baik 1,2025-08-04 00:00:00,Mon,13:36:00,fb,paid,Web Ads,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,7.0,INV-250804R2C2K,Isep Kamiludin,Bapak,100000.0,528.0,100528.0,87772391069.0,NaN,Semoga Allah mengabulkqn doa kita semua..aamiin,BANTU PALESTINA SEKARANG,instant,https://m.dana.id/s/49unkdhk,DANA,Waiting,1005.0,Teman Baik 1,2025-08-04 00:00:00,Mon,13:36:00,fb,paid,Web Ads,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,55.0,INV-2508049SZY8,A.nabila,Ibu,100000.0,589.0,100589.0,812812335.0,NaN,NaN,SEDEKAH PALESTINA,instant,https://flip.id/pwf/transaction/consolidated?r...,QRIS,Waiting,1006.0,Teman Baik 1,2025-08-04 00:00:00,Mon,10:52:00,fb,paid,Web Ads,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,57.0,INV-250804LQMVG,A.nabila,Ibu,100000.0,209.0,100209.0,812812335.0,NaN,NaN,SEDEKAH PALESTINA,transfer,BSI 7772526274,BSI,Waiting,1002.0,Teman Baik 1,2025-08-04 00:00:00,Mon,10:49:00,fb,paid,Web Ads,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.

In [25]:
data_all = data_all.dropna(subset=["Invoice ID"])

In [26]:
data_all.to_excel("Data_transaksi.xlsx")

In [27]:
data_all['Whatsapp'] = pd.to_numeric(data_all['Whatsapp'], errors='coerce')
data_all = data_all.dropna(subset=['Whatsapp'])
data_all.info()

C:\Users\faruq\AppData\Local\Temp\ipykernel_4420\1463130004.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_all['Whatsapp'] = pd.to_numeric(data_all['Whatsapp'], errors='coerce')


<class 'pandas.core.frame.DataFrame'>
Index: 110284 entries, 5 to 103652
Columns: 108 entries, Unnamed: 0.72 to UTM ID
dtypes: datetime64[ns](1), float64(80), object(27)
memory usage: 91.7+ MB


In [28]:
# change data type and make a new necessary columns 
data_all['Whatsapp'] = data_all['Whatsapp'].astype(int)
data_all['Time'] = pd.to_datetime(data_all['Time'], format='%H:%M:%S')  # ← keep full datetime
data_all['hour'] = data_all['Time'].dt.hour
data_all['label_jam'] = data_all['hour'].fillna(0).apply(
    lambda h: f"{int(h):02d}-{(int(h) + 1) % 24:02d}"
)

In [29]:
# Standarize whatsapp number
def is_random_number(num):
    num = str(num)

    # Nomor harus hanya angka
    if not num.isdigit():
        return False

    # Harus valid dari segi panjang dan awalan
    starts_valid = num.startswith("08") or num.startswith("8") or num.startswith("62")
    length_valid = 8 < len(num) <= 14

    if not (starts_valid and length_valid):
        return False

    # Cek jika terlalu banyak angka sama (seperti 08888888888)
    if len(set(num)) <= 3:
        return False

    # Cek pola berulang (seperti 081234567812345678)
    if len(num) > 6 and num[:int(len(num)/2)] == num[int(len(num)/2):]:
        return False

    # Cek pola urutan (seperti 08123456789)
    if num[2:].isdigit() and num[2:] in '1234567890'*2:
        return False

    return True

data_all['is_random'] = data_all['Whatsapp'].apply(is_random_number)
data_all['kategori_nomor'] = data_all['is_random'].map({True: 'Nomor Acak', False: 'Nomor Pola/Tidak Valid'})
data_all[data_all['is_random'] == False].head()

,Unnamed: 0.72,Unnamed: 0.71,Unnamed: 0.70,Unnamed: 0.69,Unnamed: 0.68,Unnamed: 0.67,Unnamed: 0.66,Unnamed: 0.65,Unnamed: 0.64,Unnamed: 0.63,Unnamed: 0.62,Unnamed: 0.61,Unnamed: 0.60,Unnamed: 0.59,Unnamed: 0.58,Unnamed: 0.57,Unnamed: 0.56,Unnamed: 0.55,Unnamed: 0.54,Unnamed: 0.53,Unnamed: 0.52,Unnamed: 0.51,Unnamed: 0.50,Unnamed: 0.49,Unnamed: 0.48,Unnamed: 0.47,Unnamed: 0.46,Unnamed: 0.45,Unnamed: 0.44,Unnamed: 0.43,Unnamed: 0.42,Unnamed: 0.41,Unnamed: 0.40,Unnamed: 0.39,Unnamed: 0.38,Unnamed: 0.37,Unnamed: 0.36,Unnamed: 0.35,Unnamed: 0.34,Unnamed: 0.33,Unnamed: 0.32,Unnamed: 0.31,Unnamed: 0.30,Unnamed: 0.29,Unnamed: 0.28,Unnamed: 0.27,Unnamed: 0.26,Unnamed: 0.25,Unnamed: 0.24,Unnamed: 0.23,Unnamed: 0.22,Unnamed: 0.21,Unnamed: 0.20,Unnamed: 0.19,Unnamed: 0.18,Unnamed: 0.17,Unnamed: 0.16,Unnamed: 0.15,Unnamed: 0.14,Unnamed: 0.13,Unnamed: 0.12,Unnamed: 0.11,Unnamed: 0.10,Unnamed: 0.9,Unnamed: 0.8,Unnamed: 0.7,Unnamed: 0.6,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,No,Invoice ID,Donatur,Sapaan,Nominal,Kode Unik,Total,Whatsapp,Email,Comment,Program,Payment Method,Payment Number,Payment Account,Payment Status,Fundraiser Commission,Fundraiser Name,Date,Day,Time,UTM Source,UTM Medium,Source,ip,Fundraiser Commision,aff_code,zisco,Data Qurban,Data Package-2,Data Zakat Fitrah,Additional Data,UTM Content,UTM Campaign,UTM Term,UTM ID,hour,label_jam,is_random,kategori_nomor
7218,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,508.0,INV-250720QFDBQ,Hamba Alloh,Bapak,50000.0,782.0,50782.0,81000000,NaN,Aamiin,INFAK PALESTINA: BENTUK PEDULI KITA KEPADA SAU...,transfer,Mandiri 1410000665448,Mandiri,Waiting,508.0,Teman Baik 1,2025-07-20,Sun,1900-01-01 04:37:00,ig,paid,Web Ads,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,04-05,False,Nomor Pola/Tidak Valid
27138,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,494.0,INV-2505242MJ04,Rafli Firyal,Bapak,100000.0,547.0,103877.0,87777777777,NaN,NaN,INFAK PALESTINA,va,1900800023227110,BCA VA,Success,1039.0,Teman Baik 1,2025-05-24,Sat,1900-01-01 02:11:00,ig,paid,Web Ads,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,02-03,False,Nomor Pola/Tidak Valid
27226,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,582.0,INV-2505234S3PA,Hamba Allah,Ibu,100000.0,169.0,100169.0,123506328,NaN,Bismillah hamba donasi karena Allah Barokallah...,INFAK PALESTINA,transfer,BSI 7772526274,BSI,Waiting,1002.0,Teman Baik 1,2025-05-23,Fri,1900-01-01 13:12:00,fb,paid,Web Ads,NaN,NaN,NaN

In [30]:
# Standarisasi nomor agar diawali dengan 62
def format_nomor(nomor):
    nomor = str(nomor)
    if nomor.startswith('6'):
        return nomor
    elif nomor.startswith('8'):
        return '62' + nomor
    else:
        return nomor  # Bisa disesuaikan jika ada kondisi lain
    
data_all['Whatsapp'] = data_all['Whatsapp'].apply(format_nomor)

# Rekap himpunan

In [31]:
sheet_ids = ["17mDDgZm5jrbOUZU2rhFMiC999FvhJuTtvooqs5gZAaY","1e1uyBU_1MtbdktSrnqHj-f83t7DM17S8_LFGMqYcccQ","1CUC2hdiJ2CkKDv3d5vmLdtEI4aawz7ERV7E0ChwWjxM","12ARtZE3RVK87uKPvzNbPxqouzZnEB3cAtI6WVv3emiA", "1LmfIz2ZARiROFnSOpiG1oRxYI_hV0ZdWPxeW1LJT1c8","10BLcY2mY904pTu5uAeKyFs4UhI6WAyz68zcW70Odw28", "1hGYcODnzpNyzYfLiGE4UU8SNx7yV12NqmOOpEVthtC8", "1vvRzfvm64b5yNLzt_4FKO2XpJp25vV66s8tbr3F3InE","1lSffBtyrLEa5qkoqzVdHxYCSqprpzM6EV4sGmIp0aC0", "1mfsclvlZsC4Vy3UFkNQQBGXOcFAdwsrFu5xwsIio2Vg", "1Fr8RWWtD210Mtb6Z3yykV8qvLTwPazZbd0BBKxzCkbc", "1BGLbwBVfthAIpW8p5qe4YOeOxlzI3eQx5wYDLEnxhFg", "1cvNtUmpMYosHN_8v5YgcVaALwfj2mbcaiTPoJ-sl5Vg", "16gTATflTcMEemIwNkpxQkehAzsQy-Xe-rfl3xRjU07A", "1YM8yyKBEDZgljEqivdO-6X2C1S-Sj4TjUJgZIThmL_Y"]

import gspread
from google.oauth2.service_account import Credentials

SCOPES = ["https://www.googleapis.com/auth/spreadsheets.readonly"]

creds = Credentials.from_service_account_file(
    "teman-baik-123-7e2148d71767.json",
    scopes=SCOPES
)

client = gspread.authorize(creds)

def read_sheet_by_id(sheet_id, worksheet="Rekap_Himpunan"):
    sh = client.open_by_key(sheet_id)
    ws = sh.worksheet(worksheet)

    records = ws.get_all_records()
    df = pd.DataFrame(records)

    df["source_sheet_id"] = sheet_id
    return df

dfs = []

for sid in sheet_ids:
    try:
        dfs.append(read_sheet_by_id(sid))
        time.sleep(0.5)
    except Exception as e:
        print(f"Failed to read {sid}: {e}")
        time.sleep(0.5)

rekap_himpunan = pd.concat(dfs, ignore_index=True)


rekap_himpunan['Total'] = rekap_himpunan['Total'].fillna(rekap_himpunan[' Total'])

In [32]:
# load processed data from web
# data_with_rekapan = pd.read_excel("checkpoint_files/data_with_updated_himpunan.xlsx")

# gsheet id each CRM rekap himpunan

# cs_zein = "17mDDgZm5jrbOUZU2rhFMiC999FvhJuTtvooqs5gZAaY"
# cs_vicky = "1e1uyBU_1MtbdktSrnqHj-f83t7DM17S8_LFGMqYcccQ"
# cs_intan = "1CUC2hdiJ2CkKDv3d5vmLdtEI4aawz7ERV7E0ChwWjxM"
# cs_diah = "12ARtZE3RVK87uKPvzNbPxqouzZnEB3cAtI6WVv3emiA"
# cs_shania = "1LmfIz2ZARiROFnSOpiG1oRxYI_hV0ZdWPxeW1LJT1c8"
# cs_firda = "10BLcY2mY904pTu5uAeKyFs4UhI6WAyz68zcW70Odw28"
# cs_endah = "1hGYcODnzpNyzYfLiGE4UU8SNx7yV12NqmOOpEVthtC8"
# cs_erni = "1vvRzfvm64b5yNLzt_4FKO2XpJp25vV66s8tbr3F3InE"
# cs_agil = "1lSffBtyrLEa5qkoqzVdHxYCSqprpzM6EV4sGmIp0aC0"
# cs_oliv = "1X9Sv4LbRHLFv5LLzKhPxlRHWUB_vA6byxdoWUNFE1Gw"


# rekap himpunan
# rekap_himpunan_wa_Juni = pd.read_excel("../data_blast/rekap_himpunan/old/Rekap_Himpunan_Juni.xlsx")
# rekap_himpunan_wa_7778 = pd.read_excel("../data_blast/rekap_himpunan/old/Rekap_Himpunan_7778.xlsx")
# rekap_himpunan_wa_7778_Juli = pd.read_excel("../data_blast/rekap_himpunan/old/Rekap_Himpunan_Juli.xlsx")
# rekap_himpunan_wa_2061 = pd.read_excel("../data_blast/rekap_himpunan/old/Rekap_Himpunan_2061.xlsx")
# rekap_himpunan_wa_2062 = pd.read_excel("../data_blast/rekap_himpunan/old/Rekap_Himpunan_2062.xlsx")
# rekap_himpunan_wa_2060_juli = pd.read_excel("../data_blast/rekap_himpunan/old/Rekap_Himpunan_2060_Juli.xlsx")

# # agus
# rekap_himpunan_wa_2060_agustus = pd.read_excel("../data_blast/rekap_himpunan/old/Rekap_Himpunan_2060_Agustus.xlsx")
# rekap_himpunan_0056_0036_agustus = pd.read_excel("../data_blast/rekap_himpunan/old/Rekap_Himpunan_Agustus zein.xlsx")

# # sept
# rekap_himpunan_wa_2060_september = pd.read_excel("../data_blast/rekap_himpunan/old/Rekap_Himpunan_2060_September.xlsx")
# rekap_himpunan_2056_2057_sept = pd.read_excel("../data_blast/rekap_himpunan/new/Rekap_Himpunan_2056-2057_Sept.xlsx")
# rekap_himpunan_0056_0036_sept = pd.read_excel("../data_blast/rekap_himpunan/old/Rekap_Himpunan_September zein.xlsx")
# rekap_himpunan_7778_sept = pd.read_excel("../data_blast/rekap_himpunan/new/REKAP HIMPUNAN 7778_Sept.xlsx")
# rekap_himpunan_1097_sept = pd.read_excel("../data_blast/rekap_himpunan/new/REKAP HIMPUNAN 0038_Sept.xlsx")

# # okt
# rekap_himpunan_0046_0047_okt = pd.read_excel("../data_blast/rekap_himpunan/new/Rekap Himpunan Shania Oktober.xlsx")
# rekap_himpunan_0041_0051_okt = pd.read_excel("../data_blast/rekap_himpunan/new/Rekap Himpunan Diah Oktober.xlsx")
# rekap_himpunan_2056_2057_okt = pd.read_excel("../data_blast/rekap_himpunan/new/Himpunan 2056-2057 (okt).xlsx")
# rekap_himpunan_wa_2060_oktober = pd.read_excel("../data_blast/rekap_himpunan/old/Rekap_Himpunan_2060_Oktober.xlsx")
# rekap_himpunan_1097_okt = pd.read_excel("../data_blast/rekap_himpunan/new/Rekap Himpunan Oktiber 1097.xlsx")
# rekap_himpunan_7778_okt = pd.read_excel("../data_blast/rekap_himpunan/new/Rekap_Himpunan_Oktober_7778.xlsx")
# rekap_himpunan_31_32_okt = pd.read_excel("../data_blast/rekap_himpunan/new/Rekap Himpunan Erni Oktober.xlsx")
# rekap_himpunan_1095_1096_okt = pd.read_excel("../data_blast/rekap_himpunan/new/Himpunan Firda Oktober.xlsx")
# rekap_himpunan_2058_2059_okt = pd.read_excel("../data_blast/rekap_himpunan/new/Rekap Himpunan Endah Oktober.xlsx")
# rekap_himpunan_0034_8196_okt = pd.read_excel("../data_blast/rekap_himpunan/new/Rekap Himpunan Agil Oktober.xlsx")
# rekap_himpunan_0056_0036_okt = pd.read_excel("../data_blast/rekap_himpunan/new/Rekap_Himpunan_Oktober zein.xlsx")

# # nov
# rekap_himpunan_0046_0047_nov = pd.read_excel("../data_blast/rekap_himpunan/new/Rekap Himpunan Shania November.xlsx")
# rekap_himpunan_0041_0051_nov = pd.read_excel("../data_blast/rekap_himpunan/new/Rekap Himpunan Diah November.xlsx")
# rekap_himpunan_2056_2057_nov = pd.read_excel("../data_blast/rekap_himpunan/new/Himpunan 2056-2057 (nov).xlsx")
# rekap_himpunan_31_32_nov = pd.read_excel("../data_blast/rekap_himpunan/new/Rekap Himpunan Erni November.xlsx")
# rekap_himpunan_1095_1096_nov = pd.read_excel("../data_blast/rekap_himpunan/new/Himpunan Firda November.xlsx")
# rekap_himpunan_2058_2059_nov = pd.read_excel("../data_blast/rekap_himpunan/new/Rekap Himpunan Endah November.xlsx")
# rekap_himpunan_0034_8196_nov = pd.read_excel("../data_blast/rekap_himpunan/new/Rekap Himpunan Agil November.xlsx")
# rekap_himpunan_0056_0036_nov = pd.read_excel("../data_blast/rekap_himpunan/new/Rekap_Himpunan_Cut_Off_Nopember.xlsx")
# rekap_himpunan_8194_8195_nov = pd.read_excel("../data_blast/rekap_himpunan/new/Himpunan Oliv November.xlsx")
# rekap_himpunan_1097_nov = pd.read_excel("../data_blast/rekap_himpunan/new/REKAP HIMPUNAN NOVEMBER 1097.xlsx")
# rekap_himpunan_7778_nov = pd.read_excel("../data_blast/rekap_himpunan/new/REKAP HIMPUNAN NOVEMBER 7778.xlsx")



# # collected himpunan each CRM
# rekap_himpunan_wa_2060 = pd.concat([rekap_himpunan_wa_2060_juli,rekap_himpunan_wa_2060_agustus,rekap_himpunan_wa_2060_september,rekap_himpunan_wa_2060_oktober])
# rekap_himpunan_wa_0056_0036 = pd.concat([rekap_himpunan_0056_0036_agustus, rekap_himpunan_0056_0036_sept, rekap_himpunan_0056_0036_okt, rekap_himpunan_8194_8195_nov])
# rekap_himpunan_wa_2056_2057 = pd.concat([rekap_himpunan_2056_2057_sept,rekap_himpunan_2056_2057_okt, rekap_himpunan_2056_2057_nov])
# rekap_himpunan_wa_7778_1097 = pd.concat([rekap_himpunan_wa_7778, rekap_himpunan_wa_7778_Juli,rekap_himpunan_7778_sept,rekap_himpunan_1097_sept, rekap_himpunan_7778_okt,rekap_himpunan_1097_okt, rekap_himpunan_7778_nov,rekap_himpunan_1097_nov])
# rekap_himpunan_wa_41_51 = pd.concat([rekap_himpunan_0041_0051_okt, rekap_himpunan_0041_0051_nov])
# rekap_himpunan_wa_46_47 = pd.concat([rekap_himpunan_0046_0047_okt,rekap_himpunan_0046_0047_nov])
# rekap_himpunan_wa_1095_1096 = pd.concat([rekap_himpunan_1095_1096_okt, rekap_himpunan_1095_1096_nov])
# rekap_himpunan_wa_2058_2059 = pd.concat([rekap_himpunan_2058_2059_okt, rekap_himpunan_2058_2059_nov])
# rekap_himpunan_wa_31_32 = pd.concat([rekap_himpunan_31_32_okt, rekap_himpunan_31_32_nov])
# rekap_himpunan_wa_0034_8196 = pd.concat([rekap_himpunan_0034_8196_okt, rekap_himpunan_0034_8196_nov])
# rekap_himpunan_8194_8195 = pd.concat([rekap_himpunan_8194_8195_nov])




# # all rekap himpunan
# rekap_himpunan = pd.concat([rekap_himpunan_wa_2061,rekap_himpunan_wa_2062, rekap_himpunan_wa_Juni,
#                             rekap_himpunan_wa_2060, 
#                             rekap_himpunan_wa_0056_0036,
#                             rekap_himpunan_wa_2056_2057,
#                             rekap_himpunan_wa_7778_1097,
#                             rekap_himpunan_wa_41_51,
#                             rekap_himpunan_wa_46_47,
#                             rekap_himpunan_wa_1095_1096,
#                             rekap_himpunan_wa_2058_2059,
#                             rekap_himpunan_wa_31_32,
#                             rekap_himpunan_wa_0034_8196,
#                             rekap_himpunan_8194_8195])


rekap_himpunan['Source'] = "Web Ads"


rekap_himpunan = rekap_himpunan.dropna(subset=['Total'])
rekap_himpunan['Date'] = rekap_himpunan['Date'].fillna(rekap_himpunan['Date Blast']).fillna(rekap_himpunan['Date Donation'])
rekap_himpunan['Date'] = rekap_himpunan["Date"] = pd.to_datetime(rekap_himpunan["Date"], format="%m/%d/%Y")
rekap_himpunan = rekap_himpunan[rekap_himpunan['Whatsapp'] != ""]

rekap_himpunan = rekap_himpunan[['Donatur','Whatsapp','Payment Account','Date','Total']]

In [33]:
# CTWA
folder_path = '../data_blast/rekap_himpunan/old/Oktober'

all_files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]

# Read and combine
ctwa_raw = pd.concat(
    [pd.read_excel(os.path.join(folder_path, f)) for f in all_files],
    ignore_index=True
)
ctwa_raw['Source'] = "CTWA"

c:\Users\faruq\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\faruq\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\faruq\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\faruq\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\faruq\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\faruq\AppDa

In [34]:
# rekap_himpunan_all['Whatsapp'] = rekap_himpunan_all['Whatsapp'].replace(['', 'NaN', 'None'], pd.NA)
# Drop rows where Whatsapp is null
ctwa_all = ctwa_raw[['CRM','Tanggal','Nama Donatur','Whatsapps','Payment Status','Bank','Donasi','Akad','Source']].dropna(subset=['Donasi'])
# rekap_himpunan_all
ctwa = ctwa_all.rename(columns={'Whatsapps':'Whatsapp', 'Nama Donatur':'Donatur','Donasi':'Total','Akad':'Program','Tanggal':'Date'})

In [35]:
# make sure whatsapp dtype is the same (int)
ctwa['Whatsapp'] = ctwa['Whatsapp'].astype(int)
rekap_himpunan['Whatsapp'] = rekap_himpunan['Whatsapp'].fillna(0).astype(int)

# concat ctwa and rekap himpunan
rekap_himpunan_all = pd.concat([ctwa,rekap_himpunan])

C:\Users\faruq\AppData\Local\Temp\ipykernel_4420\3653407071.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  rekap_himpunan['Whatsapp'] = rekap_himpunan['Whatsapp'].fillna(0).astype(int)


In [36]:
rekap_himpunan_all['Date'] = rekap_himpunan_all['Date'].apply(
    lambda x: pd.to_datetime(x, errors='coerce') if not isinstance(x, pd.Timestamp) else x
)

# choose only necessary columns
rekap_himpunan_all = rekap_himpunan_all[['Donatur', 'Whatsapp', 'Payment Account', 'Date', 'Program',
       'Total', 'Source']]

# get a day column
rekap_himpunan_all['Day'] = rekap_himpunan_all['Date'].dt.strftime('%a')

Standarisasi Payment Account

In [37]:
def payment(akun,number):
    number = str(number)
    if akun == "Yayasan Teman Jalan Kebaikan":
        if ' ' not in number:
            return "BSI"
        else: 
            return number.split()[0]
    else:
        return akun
    
data_all['Payment'] = data_all.apply(lambda row: payment(row['Payment Account'], row['Payment Number']), axis=1)

data_all.drop('Payment Account', axis=1, inplace=True)

In [38]:
def move_column(df, col_name, new_pos):
    cols = list(df.columns)
    cols.insert(new_pos, cols.pop(cols.index(col_name)))
    return df[cols]

data_all = move_column(data_all,"Payment", 13)


data_all.rename(columns={'Payment': 'Payment Account'}, inplace=True)

In [39]:
df = data_all[(data_all['is_random'] == True)]
# df = df[df['Payment Status'] == "Success"]
# df = data_all[data_all['Payment Status'] == "Success"]

Menggabungkan data dari web dengan rekap himpunan

In [40]:
rekap_himpunan_all['Program'] = rekap_himpunan_all['Program'].str.upper()

rekap_himpunan_all['Whatsapp'] = rekap_himpunan_all['Whatsapp'].astype(str)

rekap_himpunan_all = rekap_himpunan_all.drop_duplicates()

In [41]:
df = df.loc[:, ~df.columns.str.contains("Unnamed")]
df

,Payment Account,No,Invoice ID,Donatur,Sapaan,Nominal,Kode Unik,Total,Whatsapp,Email,Comment,Program,Payment Method,Payment Number,Payment Status,Fundraiser Commission,Fundraiser Name,Date,Day,Time,UTM Source,UTM Medium,Source,ip,Fundraiser Commision,aff_code,zisco,Data Qurban,Data Package-2,Data Zakat Fitrah,Additional Data,UTM Content,UTM Campaign,UTM Term,UTM ID,hour,label_jam,is_random,kategori_nomor
5,DANA,6.0,INV-2508040IVSI,Mayang,Bapak,100000.0,446.0,100446.0,6282162426297,NaN,NaN,INFAK PALESTINA: BENTUK PEDULI KITA KEPADA SAU...,instant,https://m.dana.id/s/m7v7pwbn,Waiting,1004.0,Teman Baik 1,2025-08-04 00:00:00,Mon,1900-01-01 13:36:00,fb,paid,Web Ads,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,13-14,True,Nomor Acak
6,DANA,7.0,INV-250804R2C2K,Isep Kamiludin,Bapak,100000.0,528.0,100528.0,6287772391069,NaN,Semoga Allah mengabulkqn doa kita semua..aamiin,BANTU PALESTINA SEKARANG,instant,https://m.dana.id/s/49unkdhk,Waiting,1005.0,Teman Baik 1,2025-08-04 00:00:00,Mon,1900-01-01 13:36:00,fb,paid,Web Ads,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,13-14,True,Nomor Acak
54,QRIS,55.0,INV-2508049SZY8,A.nabila,Ibu,100000.0,589.0,100589.0,62812812335,NaN,NaN,SEDEKAH PALESTINA,instant,https://flip.id/pwf/transaction/consolidated?r...,Waiting,1006.0,Teman Baik 1,2025-08-04 00:00:00,Mon,1900-01-01 10:52:00,fb,paid,Web Ads,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,10-11,True,Nomor Acak
56,BSI,57.0,INV-250804LQMVG,A.nabila,Ibu,100000.0,209.0,100209.0,62812812335,NaN,NaN,SEDEKAH PALESTINA,transfer,BSI 7772526274,Waiting,1002.0,Teman Baik 1,2025-08-04 00:00:00,Mon,1900-01-01 10:49:00,fb,paid,Web Ads,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,10-11,True,Nomor Acak
72,GOPAY,73.0,INV-250804DZ01S,Didi sutardi.didi1968@gmail.com,Bapak,50000.0,640.0,50640.0,6281324233883,didisutardi0568@gmail.com,Semoga Allah cepat akhiri penderitaan Rakyat d...,BANTU PALESTINA SEKARANG,instant,https://api.midtrans.com/v2/gopay/d1730895-e65...,Waiting,506.0,Teman Baik 1,2025-08-04 00:00:00,Mon,1900-01-01 10:12:00,fb,paid,Web Ads,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,10-11,True,Nomor Acak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103648,BRI,NaN,INV-260107X7Q88,Muhammad Junedi Hairul Syah,NaN,NaN,NaN,100800.0,6285342000732,,,BERSAMA RINGANKAN DERITA SUMATERA,transfer,BRI 115901000647561,Success,NaN,Teman Baik 1,2026-01-07 13:55:40,Wed,1900-01-01 13:55:40,fb,Facebook_Mobile_Reels,Web Ads,36.74.123.215,1008.0,iuny9,NaN,[],[],[],"{""Kode Unik"":""800""}",BPK LPR,BPK LPR,120242969586980322,120242969586970322,13,13-14,True,Nomor Acak
103649,Mandiri,NaN,INV-260107MMW3P,Tetie suratmi,NaN,NaN,NaN,250807.0,6285240130373,,Smoga Allah melindungi kalian semua yg terdam...,BERSAMA RINGANKAN DERITA SUMATERA,transfer,Mandiri 1410000665448,Success,NaN,Teman Baik 1,2026-01-07 13:57:56,Wed,1900-01-01 13:57:56,fb,Facebook_Mobile_Feed,Web Ads,180.243.35.187,2508.0,iuny9,NaN,[],[],[],"{""Kode Unik"":""807""}",BPK LPR,BPK LPR,120242969586980322,120242969586970322,13,13-14,True,Nomor Acak
103650,BCA VA,NaN,INV-2601071NZ1F,Hamba Allah,NaN,NaN,NaN,104213.0,628161994665,,Semoga manfaat dan barokah,BERSAMA RINGANKAN DERITA SUMATERA,va,1900800027042947,Success,NaN,Teman Baik 1,2026-01-07 14:26:27,Wed,1900-01-01 14:26:27,fb,Facebook_Mobile_Reels,Web Ads,118.99.81.168,1042.0,iuny9,NaN,[],[],[],"{""Kode Unik"":""883""}",BPK LPR,BPK LPR,120242969586980322,120242969586970322,14,14-15,True,Nomor Acak
103651,BCA VA,NaN,INV-260107P4ZQ5,Lilis,NaN,NaN,NaN,103440.0,6281217158189,,Bismillah yg terkena musibah diberi ketabahan ...,BERSAMA RINGANKAN DERITA SUMATERA,va,1900800027042980,Success,NaN,Teman Baik 1,2026-01-07 14:29:35,Wed,1900-01-01 14:29:35,ig,Instagram_Reels,Web Ads,103.67.85.34,1034.0,iuny9,NaN,[],[],[],"{""Kode Unik"":""110""}",BPK LPR,BPK LPR,120242969586980322,120242969586970322,14,14-15,True,Nomor Acak


In [42]:
import re

data_web_himpunan = pd.concat([df,rekap_himpunan_all])

data_web_himpunan['Total'] = data_web_himpunan['Total'].apply(
    lambda x: int(re.sub(r'[^0-9]', '', x)) if isinstance(x, str) else x
)

In [43]:
data_web_himpunan['Payment Status'] = data_web_himpunan['Payment Status'].fillna("Success")

# Format tanggal dan ambil bulan
data_web_himpunan["Tanggal"] = pd.to_datetime(data_web_himpunan["Date"], dayfirst=True)
data_web_himpunan["Bulan"] = data_web_himpunan["Date"].dt.month
data_web_himpunan["Tahun"] = data_web_himpunan["Date"].dt.year
data_web_himpunan["Bulan_Nama"] = data_web_himpunan["Date"].dt.strftime('%B')

In [44]:
# Choose only success donation
first_donation_success = data_web_himpunan[data_web_himpunan['Payment Status'] == "Success"]

# Bulan pertama donasi
first_donation = first_donation_success.groupby("Whatsapp")["Date"].min().reset_index()

In [45]:
first_donation["Bulan_Pertama"] = first_donation["Date"].dt.month
first_donation["Tahun_Pertama"] = first_donation["Date"].dt.year

# Gabung ke df utama
data_web_himpunan = data_web_himpunan.merge(first_donation[["Whatsapp", "Bulan_Pertama"]], on="Whatsapp")
data_web_himpunan = data_web_himpunan[data_web_himpunan["Bulan"] >= data_web_himpunan["Bulan_Pertama"]]

In [46]:
data_web_himpunan

,Payment Account,No,Invoice ID,Donatur,Sapaan,Nominal,Kode Unik,Total,Whatsapp,Email,Comment,Program,Payment Method,Payment Number,Payment Status,Fundraiser Commission,Fundraiser Name,Date,Day,Time,UTM Source,UTM Medium,Source,ip,Fundraiser Commision,aff_code,zisco,Data Qurban,Data Package-2,Data Zakat Fitrah,Additional Data,UTM Content,UTM Campaign,UTM Term,UTM ID,hour,label_jam,is_random,kategori_nomor,Tanggal,Bulan,Tahun,Bulan_Nama,Bulan_Pertama
0,DANA,7.0,INV-250804R2C2K,Isep Kamiludin,Bapak,100000.0,528.0,100528.0,6287772391069,NaN,Semoga Allah mengabulkqn doa kita semua..aamiin,BANTU PALESTINA SEKARANG,instant,https://m.dana.id/s/49unkdhk,Waiting,1005.0,Teman Baik 1,2025-08-04,Mon,1900-01-01 13:36:00,fb,paid,Web Ads,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13-14,True,Nomor Acak,2025-08-04,8,2025,August,5
1,QRIS,55.0,INV-2508049SZY8,A.nabila,Ibu,100000.0,589.0,100589.0,62812812335,NaN,NaN,SEDEKAH PALESTINA,instant,https://flip.id/pwf/transaction/consolidated?r...,Waiting,1006.0,Teman Baik 1,2025-08-04,Mon,1900-01-01 10:52:00,fb,paid,Web Ads,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,10-11,True,Nomor Acak,2025-08-04,8,2025,August,7
2,BSI,57.0,INV-250804LQMVG,A.nabila,Ibu,100000.0,209.0,100209.0,62812812335,NaN,NaN,SEDEKAH PALESTINA,transfer,BSI 7772526274,Waiting,1002.0,Teman Baik 1,2025-08-04,Mon,1900-01-01 10:49:00,fb,paid,Web Ads,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,10-11,True,Nomor Acak,2025-08-04,8,2025,August,7
3,GOPAY,73.0,INV-250804DZ01S,Didi sutardi.didi1968@gmail.com,Bapak,50000.0,640.0,50640.0,6281324233883,didisutardi0568@gmail.com,Semoga Allah cepat akhiri penderitaan Rakyat d...,BANTU PALESTINA SEKARANG,instant,https://api.midtrans.com/v2/gopay/d1730895-e65...,Waiting,506.0,Teman Baik 1,2025-08-04,Mon,1900-01-01 10:12:00,fb,paid,Web Ads,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,10-11,True,Nomor Acak,2025-08-04,8,2025,August,8
4,BSI,109.0,INV-250804XSS8E,Hj.Norminah,Bapak,100000.0,289.0,100289.0,6282255266762,umianis610@gmail.com,bismillah selalu d beri kesabaran,INFAK PALESTINA: BENTUK PEDULI KITA KEPADA SAU...,transfer,BSI 7772526274,Waiting,1003.0,Teman Baik 1,2025-08-04,Mon,1900-01-01 08:53:00,ig,paid,Web Ads,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,08-09,True,Nomor Acak,2025-08-04,8,2025,August,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96654,BNI (CS) 5005003227,NaN,NaN,Sugiyanto,NaN,NaN,NaN,100000.0,628156537606,NaN,NaN,NaN,NaN,NaN,Success,NaN,NaN,2025-12-11,Thu,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-12-11,12,2025,December,12
96655,BNI (CS) 5005003227,NaN,NaN,Arbainah,NaN,NaN,NaN,15000.0,6285731003398,NaN,NaN,NaN,NaN,NaN,Success,NaN,NaN,2025-12-06,Sat,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-12-06,12,2025,December,12
96656,Mandiri (CS) 1410029052222,NaN,NaN,Busadli,NaN,NaN,NaN,50000.0,6285278333738,NaN,NaN,NaN,NaN,NaN,Success,NaN,NaN,2025-12-18,Thu,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-12-18,12,2025,December,12
96657,BRI (CS) 115901000648567,NaN,NaN,Yurlianis,NaN,NaN,NaN,100000.0,6285274087160,NaN,NaN,NaN,NaN,NaN,Success,NaN,NaN,2025-12-31,Wed,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-12-31,12,2025,December,12


# Tracking

In [47]:
tracking_rekapan = pd.read_excel("checkpoint_files/data_with_updated_himpunan.xlsx")

In [48]:
data_success_only = data_web_himpunan[data_web_himpunan['Payment Status'] == "Success"]
data_success_only = data_success_only.sort_values('Date')


In [49]:
agg_success = data_success_only.groupby(["Whatsapp", "Bulan"]).agg(
    Donatur=("Donatur", "last"),
    Total=("Total", "sum"),
    Date=("Date", "min"),
    Payment=("Payment Account", "first"),
    Program=("Program", lambda x: list(x)),
    Time=('label_jam', lambda x: list(x)),
    Tanggal_terakhir_donasi=("Tanggal", max),
    Day=("Day", lambda x: x.mode().iloc[0] if not x.mode().empty else None)
).reset_index()

all_donor = data_web_himpunan[["Whatsapp", "Bulan"]].drop_duplicates()

all_agg = all_donor.merge(agg_success, on=["Whatsapp", "Bulan"], how="left")

C:\Users\faruq\AppData\Local\Temp\ipykernel_4420\3681626778.py:1: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  agg_success = data_success_only.groupby(["Whatsapp", "Bulan"]).agg(


In [50]:
final_programs = []
previous_programs = {}

for _, row in all_agg.iterrows():
    key = row["Whatsapp"]
    val = row["Program"]

    # --- Normalisasi Program agar selalu list (no NaN mix) --- #
    if isinstance(val, float) and pd.isna(val):
        current_programs = []
    elif isinstance(val, list):
        current_programs = [x for x in val if not (isinstance(x, float) and pd.isna(x))]
        current_programs = list(dict.fromkeys(current_programs))
    else:
        current_programs = [val]

    prev = previous_programs.get(key, [])

    different_programs = [p for p in current_programs if p not in prev]

    if not prev:
        final_programs.append(current_programs[0] if current_programs else None)
    elif different_programs:
        final_programs.append(different_programs[0])
    else:
        final_programs.append(current_programs[0] if current_programs else None)

    previous_programs[key] = current_programs

In [51]:
from collections import Counter
import pandas as pd
import ast
import math

def custom_mode_from_list(time_list):
    # if the value is nan then return none
    if isinstance(time_list, float) and math.isnan(time_list):
        return None

    # change the value to list
    if not isinstance(time_list, list):
        time_list = [time_list]

    # remove nan from the list
    cleaned = [x for x in time_list if not (isinstance(x, float) and math.isnan(x))]

    if len(cleaned) == 0:
        return None
    if len(cleaned) == 1:
        return cleaned[0]

    counter = Counter(cleaned)
    mc = counter.most_common()

    # if there is more than one and the top freq is tie then return the last value
    if len(mc) > 1 and mc[0][1] == mc[1][1]:
        return cleaned[-1]

    return mc[0][0]  # mode tunggal

# if column value contains string list change to list
all_agg['Time'] = all_agg['Time'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

all_agg['Label_Jam'] = all_agg['Time'].apply(custom_mode_from_list)

In [52]:
# input the final program from the previous processing
all_agg["Final_Program"] = final_programs

# Persiapan output format lebar (wide)
bulan_dict = {
    1: "Januari", 2: "Februari", 3: "Maret", 4: "April", 5: "Mei", 6: "Juni",
    7: "Juli", 8: "Agustus", 9: "September", 10: "Oktober", 11: "November", 12: "Desember"
}

# Output awal
output_test = pd.DataFrame(all_agg["Whatsapp"].drop_duplicates())

for bulan_num in range(1, 13):
    bulan_nama = bulan_dict[bulan_num]
    sub = all_agg[all_agg["Bulan"] == bulan_num][["Whatsapp", "Final_Program", "Total", "Date", "Day", "Label_Jam", "Payment"]]
    sub.columns = ["Whatsapp", f"{bulan_nama}_Program", f"{bulan_nama}_Donasi", f"{bulan_nama}_Tanggal", f"{bulan_nama}_Day", f"{bulan_nama}_Time", f"{bulan_nama}_Payment"]
    output_test = output_test.merge(sub, on="Whatsapp", how="left")

# Tambah Nama
output_test = output_test.merge(data_web_himpunan[["Whatsapp", "Donatur"]].drop_duplicates("Whatsapp"), on="Whatsapp", how="left")

# Tambah bulan pertama donasi untuk pengurutan
output_test = output_test.merge(first_donation[["Whatsapp", "Bulan_Pertama"]], on="Whatsapp")
output_test = output_test.sort_values("Bulan_Pertama").drop(columns=["Bulan_Pertama"])

In [53]:
# Urutkan kolom
cols = output_test.columns.tolist()
cols = ["Whatsapp", "Donatur"] + [col for col in cols if col not in ["Whatsapp", "Donatur"]]
output_test = output_test[cols]

# Hapus nilai 0
output_test.replace(0, "", inplace=True)
# Ambil semua kolom yang mengandung '_Program'
program_cols = [col for col in output_test.columns if '_Program' in col]

time_cols = [col for col in output_test.columns if '_Time' in col]

# Total donasi
output_test['Total'] = output_test[[col for col in output_test.columns if '_Donasi' in col]].sum(axis=1)

# Frekuensi donasi (jumlah kolom donasi yang tidak kosong)
output_test['Frekuensi'] = output_test[[col for col in output_test.columns if '_Donasi' in col]].notna().sum(axis=1)

# output['Total_Preferensi'] = (output[[col for col in output.columns if '_Preferensi' in col]].sum(axis=1) / output[[col for col in output.columns if '_Preferensi' in col]].notna().sum(axis=1)).astype(int)

# Hitung preferensi: jumlah program unik per baris
output_test['Preferensi'] = output_test[program_cols].apply(lambda row: row.dropna().nunique(), axis=1)

# Hitung program yang paling sering muncul per baris
output_test['Program'] = output_test[program_cols].apply(lambda row: row.dropna().mode().iloc[0] if not row.dropna().empty else None, axis=1)

# Hitung range time paling sering muncul per baris
output_test['Label_Jam'] = output_test[time_cols].apply(lambda row: row.dropna().mode().iloc[0] if not row.dropna().empty else None, axis=1)

In [54]:
# choose donation column
donasi_cols = [col for col in output_test.columns if '_Donasi' in col]

# 1) cleanse and convert every donation column to numeric(non-numeric -> NaN)
for c in donasi_cols:
    # change to string first, delete character beside digit, minus, and dot (ex: "50.000" -> "50.000", "50,000" -> "50000")
    # and also handle thousand sign and whitespace
    cleaned = output_test[c].astype(str).str.replace(r'[^\d\.\-]', '', regex=True)

    # if empty string "" after cleansing, treat it as a NaN
    cleaned = cleaned.replace('', np.nan)

    # konversi ke numeric
    output_test[c] = pd.to_numeric(cleaned, errors='coerce')

# 2) Hitung mean per baris (abaikan NaN)
mean_series = output_test[donasi_cols].mean(axis=1, skipna=True)

# 3) Bulatkan dan ubah NaN -> None, non-NaN -> int
output_test['Rata - rata'] = mean_series.round(0).apply(lambda x: None if pd.isna(x) else int(x))
# Change total dtype to int
output_test['Total'] = output_test['Total'].astype(int)

In [55]:
data_web_himpunan.sort_values('Date', inplace=True)

date_and_day = data_web_himpunan.groupby('Whatsapp').agg(
    Donasi_terakhir=("Total","last"),
    Status_payment=("Payment Status", lambda x: list(x)),
    Day_Mode=("Day", lambda x: x.mode().iloc[0]),
    Date=("Tanggal", list),
    Source=("Source","first")
).reset_index()

In [56]:
def classify_cutoff(dates):
    # Konversi ke pandas Series dan ambil hari (tanggal) saja
    days = pd.Series(dates).dt.day
    
    # Hitung modus
    modus = days.mode().iloc[0]
    
    # Klasifikasi berdasarkan rentang tanggal cut-off
    return 'Masa Gajian' if (modus >= 26 or modus <= 5) else 'Bukan Gajian'

date_and_day['Date_Category'] = date_and_day['Date'].apply(classify_cutoff)
date_and_day['Date_Category'] = np.where(date_and_day['Day_Mode'] == 'Fri', 'Jumat', date_and_day['Date_Category'])
date_and_day = date_and_day.drop(columns=['Date'])

date_and_day['Status'] = date_and_day['Status_payment'].apply(
    lambda lst: 'Pernah Sukses' if any(x == 'Success' for x in lst if pd.notna(x)) 
                else 'Tetap Waiting'
)


In [57]:
# merge date and day and also first donation
output_test = pd.merge(output_test,date_and_day, how='left', on='Whatsapp')
output_test = output_test.merge(first_donation, how='left', on='Whatsapp')

# change the year and month from date column
output_test['Bulan'] = output_test['Date'].dt.strftime('%B')
output_test['Tahun'] = output_test['Date'].dt.year
# 
output_test['klasifikasi_program'] = np.where(
    output_test['Program'].str.lower().str.contains("palestina|gaza"),
    "PALESTINA",
    np.where(
        output_test['Program'].str.lower().str.contains("sudan"),
        "SUDAN",
        np.where(output_test['Program'].str.lower().str.contains("sumatera"),
        "SUMATERA",
            output_test['Program']
        )
    )
)

In [58]:
# Tambahkan kolom 'Kategori' berdasarkan nilai 'Frekuensi'
output_test['Kategori'] = output_test['Frekuensi'].apply(
    lambda x: 'Pasif' if x < 1 else ('Aktif' if x < 4.5 else 'Loyal')
)

# Tambahkan kolom 'Badge' berdasarkan nilai 'Total'
output_test['Badge'] = output_test['Total'].apply(
    lambda x: 'BRONZE' if x < 100000 else ('SILVER' if x <= 1000000 else ('GOLD' if x < 5000000 else 'PLATINUM'))
)

# Tambahkan kolom 'Avg Kategori' berdasarkan nilai 'Rata - rata'
output_test['Avg Kategori'] = output_test['Rata - rata'].apply(
    lambda x: "< 100.000" if x < 100000 else ("100.000 - 1000.000" if x <= 1000000 else ("1000.000 - 5000.000" if x < 5000000 else "> 5000.000"))
)

In [59]:
output_test['Whatsapp'] = output_test['Whatsapp'].astype(int)

In [60]:
data_with_rekapan_updated = pandas_upsert_with_stats(
    df_main=tracking_rekapan,
    df_update=output_test,
    key="Whatsapp"
)

UPSERT SUMMARY
----------------------------------------
Rows updated        : 68018
Rows inserted       : 530
Columns added       : 1
  Added columns     : ['Status_payment']
Column changes:
  Juni_Payment    : 1
  klasifikasi_program : 21
  Day_Mode        : 4
  Preferensi      : 21
  Juni_Donasi     : 1
  Frekuensi       : 22
  November_Payment : 4
  Label_Jam       : 28
  Mei_Time        : 1
  Rata - rata     : 30
  Donatur         : 5
  Kategori        : 21
  Desember_Donasi : 3
  Juni_Day        : 1
  Program         : 22
  Desember_Payment : 4
  Oktober_Payment : 1
  Bulan           : 22
  Desember_Program : 3
  Januari_Time    : 26
  Desember_Time   : 2
  Donasi_terakhir : 58
  Desember_Tanggal : 3
  Status_payment  : 68018
  Januari_Donasi  : 26
  Date_Category   : 7
  Avg Kategori    : 22
  Januari_Program : 20
  Juni_Tanggal    : 1
  Januari_Tanggal : 20
  Bulan_Pertama   : 22
  Desember_Day    : 3
  September_Payment : 2
  Januari_Payment : 20
  Date            : 23
  Badge 

In [61]:
duplicates = data_with_rekapan_updated[data_with_rekapan_updated['Whatsapp'].duplicated(keep=False)]

if not duplicates.empty:
    print("❌ Duplicate rows found:\n", duplicates)
    raise SystemExit("Stopping cell due to duplicates.")

In [62]:
data_with_rekapan_updated = data_with_rekapan_updated[['Whatsapp', 'Donatur',
       'Januari_Program', 'Januari_Donasi', 'Januari_Tanggal', 'Januari_Day',
       'Januari_Time', 'Januari_Payment', 'Februari_Program',
       'Februari_Donasi', 'Februari_Tanggal', 'Februari_Day', 'Februari_Time',
       'Februari_Payment', 'Maret_Program', 'Maret_Donasi', 'Maret_Tanggal',
       'Maret_Day', 'Maret_Time', 'Maret_Payment', 'April_Program',
       'April_Donasi', 'April_Tanggal', 'April_Day', 'April_Time',
       'April_Payment', 'Mei_Program', 'Mei_Donasi', 'Mei_Tanggal', 'Mei_Day',
       'Mei_Time', 'Mei_Payment', 'Juni_Program', 'Juni_Donasi',
       'Juni_Tanggal', 'Juni_Day', 'Juni_Time', 'Juni_Payment', 'Juli_Program',
       'Juli_Donasi', 'Juli_Tanggal', 'Juli_Day', 'Juli_Time', 'Juli_Payment',
       'Agustus_Program', 'Agustus_Donasi', 'Agustus_Tanggal', 'Agustus_Day',
       'Agustus_Time', 'Agustus_Payment', 'September_Program',
       'September_Donasi', 'September_Tanggal', 'September_Day',
       'September_Time', 'September_Payment', 'Oktober_Program',
       'Oktober_Donasi', 'Oktober_Tanggal', 'Oktober_Day', 'Oktober_Time',
       'Oktober_Payment', 'November_Program', 'November_Donasi',
       'November_Tanggal', 'November_Day', 'November_Time', 'November_Payment',
       'Desember_Program', 'Desember_Donasi', 'Desember_Tanggal',
       'Desember_Day', 'Desember_Time', 'Desember_Payment', 'Total',
       'Frekuensi', 'Preferensi', 'Program','klasifikasi_program','Label_Jam', 'Rata - rata',
       'Day_Mode', 'Date_Category', 'Date', 'Bulan_Pertama', 'Tahun_Pertama',
       'Bulan', 'Tahun', 'Kategori','Status', 'Badge', 'Avg Kategori',"Tanggal_terakhir_donasi","Donasi_terakhir", "Source"]]

In [63]:
data_with_rekapan_updated.to_excel("checkpoint_files/data_with_updated_himpunan.xlsx", index=False)

# Check number validity

In [117]:
# data with number validity (registered and not)
data_with_validity = pd.read_excel("checkpoint_files/data_with_number_validity_updated.xlsx")
data_with_validity = data_with_validity[["Whatsapp", "validity"]]

# crm data
df_crm = pd.read_excel("checkpoint_files/data_each_crm_updated.xlsx")
df_crm = df_crm[['Whatsapp','CRM']]

# category each crm
category_update = pd.read_excel("checkpoint_files/category_each_donor.xlsx")
category_update = category_update[['Whatsapp', 'kategori_update', 'kategori_prioritas',
       'terakhir_interaksi', 'start_date','kategori_karantina']]

# make sure whatsapp dtype is int on both dataframes
df_crm['Whatsapp'] = df_crm['Whatsapp'].astype("Int64")
data_with_rekapan_updated['Whatsapp'] = data_with_rekapan_updated['Whatsapp'].astype(int)
data_with_validity['Whatsapp'] = data_with_validity['Whatsapp'].astype(int)

# merge to get a number validity
df_tracking_valid = data_with_rekapan_updated.merge(data_with_validity, how="left", on="Whatsapp")

In [118]:
df_tracking_valid = df_tracking_valid[['Whatsapp', 'Donatur', 'Bulan', 'Tahun', 'Januari_Program', 'Januari_Donasi',
       'Januari_Tanggal', 'Januari_Day', 'Januari_Time', 'Januari_Payment',
       'Februari_Program', 'Februari_Donasi', 'Februari_Tanggal',
       'Februari_Day', 'Februari_Time', 'Februari_Payment', 'Maret_Program',
       'Maret_Donasi', 'Maret_Tanggal', 'Maret_Day', 'Maret_Time',
       'Maret_Payment', 'April_Program', 'April_Donasi', 'April_Tanggal',
       'April_Day', 'April_Time', 'April_Payment', 'Mei_Program', 'Mei_Donasi',
       'Mei_Tanggal', 'Mei_Day', 'Mei_Time', 'Mei_Payment', 'Juni_Program',
       'Juni_Donasi', 'Juni_Tanggal', 'Juni_Day', 'Juni_Time', 'Juni_Payment',
       'Juli_Program', 'Juli_Donasi', 'Juli_Tanggal', 'Juli_Day', 'Juli_Time',
       'Juli_Payment', 'Agustus_Program', 'Agustus_Donasi', 'Agustus_Tanggal',
       'Agustus_Day', 'Agustus_Time', 'Agustus_Payment', 'September_Program',
       'September_Donasi', 'September_Tanggal', 'September_Day',
       'September_Time', 'September_Payment', 'Oktober_Program',
       'Oktober_Donasi', 'Oktober_Tanggal', 'Oktober_Day', 'Oktober_Time',
       'Oktober_Payment', 'November_Program', 'November_Donasi',
       'November_Tanggal', 'November_Day', 'November_Time', 'November_Payment',
       'Desember_Program', 'Desember_Donasi', 'Desember_Tanggal',
       'Desember_Day', 'Desember_Time', 'Desember_Payment', 'Total',
       'Frekuensi', 'Preferensi', 'Program','klasifikasi_program' ,'Label_Jam', 'Rata - rata',
       'Day_Mode', 'Date_Category', 'Date', 'Bulan_Pertama', 'Tahun_Pertama',
       'Kategori', 'Badge','Status', 'Avg Kategori','Tanggal_terakhir_donasi',"Donasi_terakhir", 'validity', 'Source']]

In [119]:
duplicates = df_tracking_valid[df_tracking_valid['Whatsapp'].duplicated(keep=False)]

if not duplicates.empty:
    print("❌ Duplicate rows found:\n", duplicates)
    raise SystemExit("Stopping cell due to duplicates.")

In [120]:
df_with_crm = df_tracking_valid.merge(df_crm,how='left',on='Whatsapp')

321789732189

In [121]:
df_with_crm = df_with_crm[['Whatsapp', 'Donatur',  'CRM', 'Bulan', 'Tahun', 'Januari_Program',
       'Januari_Donasi', 'Januari_Tanggal', 'Januari_Day', 'Januari_Time',
       'Januari_Payment', 'Februari_Program', 'Februari_Donasi',
       'Februari_Tanggal', 'Februari_Day', 'Februari_Time', 'Februari_Payment',
       'Maret_Program', 'Maret_Donasi', 'Maret_Tanggal', 'Maret_Day',
       'Maret_Time', 'Maret_Payment', 'April_Program', 'April_Donasi',
       'April_Tanggal', 'April_Day', 'April_Time', 'April_Payment',
       'Mei_Program', 'Mei_Donasi', 'Mei_Tanggal', 'Mei_Day', 'Mei_Time',
       'Mei_Payment', 'Juni_Program', 'Juni_Donasi', 'Juni_Tanggal',
       'Juni_Day', 'Juni_Time', 'Juni_Payment', 'Juli_Program', 'Juli_Donasi',
       'Juli_Tanggal', 'Juli_Day', 'Juli_Time', 'Juli_Payment',
       'Agustus_Program', 'Agustus_Donasi', 'Agustus_Tanggal', 'Agustus_Day',
       'Agustus_Time', 'Agustus_Payment', 'September_Program',
       'September_Donasi', 'September_Tanggal', 'September_Day',
       'September_Time', 'September_Payment', 'Oktober_Program',
       'Oktober_Donasi', 'Oktober_Tanggal', 'Oktober_Day', 'Oktober_Time',
       'Oktober_Payment', 'November_Program', 'November_Donasi',
       'November_Tanggal', 'November_Day', 'November_Time', 'November_Payment',
       'Desember_Program', 'Desember_Donasi', 'Desember_Tanggal',
       'Desember_Day', 'Desember_Time', 'Desember_Payment', 'Total',
       'Frekuensi', 'Preferensi', 'Program', 'klasifikasi_program',
       'Label_Jam', 'Rata - rata', 'Day_Mode', 'Date_Category', 'Date',
       'Bulan_Pertama', 'Tahun_Pertama', 'Kategori', 'Badge', 'Avg Kategori',
       'Tanggal_terakhir_donasi',"Donasi_terakhir", 'Status', 'validity', 'Source']]

In [122]:
df_output_final = df_with_crm.merge(category_update, how='left', on='Whatsapp')

In [123]:
def update_category(df):
    # if Kategori_y isna take Kategori_x
    if pd.isna(df['kategori_update']):
        return df['Kategori']
    
    # if Kategori_x and Kategori_y are different take Kategori_y
    elif df['kategori_update'] == "Aktif":
        return df['Kategori']
    elif df['kategori_update'] in ['Eliminasi', 'Invalid']:
        return df['kategori_update']
    
    # if both are the same take Kategori_x
    else:
        return df['Kategori']

df_output_final['Kategori'] = df_output_final.apply(update_category, axis=1)

In [124]:
month_order = ['January', 'February', 'March', 'April', 'May', 'June',
               'July', 'August', 'September', 'October', 'November', 'December']
df_output_final['Bulan'] = pd.Categorical(df_output_final['Bulan'], categories=month_order, ordered=True)
df_output_final = df_output_final.sort_values('Bulan')

In [125]:
duplicates = df_output_final[df_output_final['Whatsapp'].duplicated(keep=False)]

if not duplicates.empty:
    print("❌ Duplicate rows found:\n", duplicates)
    raise SystemExit("Stopping cell due to duplicates.")

In [126]:
df_output_final['validity'].value_counts(dropna=False)

validity
Number registered                                                                                                                                  44459
NaN                                                                                                                                                30720
Number not registered                                                                                                                               6579
Error: 504 Server Error: Gateway Time-out for url: https://api.starsender.online/api/check-number                                                     13
Error: 502 Server Error: Bad Gateway for url: https://api.starsender.online/api/check-number                                                           3
Error: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))        2
Timeout                                                                  

In [127]:
# tracking data
df_output_final.to_excel("Tracking_temanbaik.xlsx")

In [128]:
df_output_final

,Whatsapp,Donatur,CRM,Bulan,Tahun,Januari_Program,Januari_Donasi,Januari_Tanggal,Januari_Day,Januari_Time,Januari_Payment,Februari_Program,Februari_Donasi,Februari_Tanggal,Februari_Day,Februari_Time,Februari_Payment,Maret_Program,Maret_Donasi,Maret_Tanggal,Maret_Day,Maret_Time,Maret_Payment,April_Program,April_Donasi,April_Tanggal,April_Day,April_Time,April_Payment,Mei_Program,Mei_Donasi,Mei_Tanggal,Mei_Day,Mei_Time,Mei_Payment,Juni_Program,Juni_Donasi,Juni_Tanggal,Juni_Day,Juni_Time,Juni_Payment,Juli_Program,Juli_Donasi,Juli_Tanggal,Juli_Day,Juli_Time,Juli_Payment,Agustus_Program,Agustus_Donasi,Agustus_Tanggal,Agustus_Day,Agustus_Time,Agustus_Payment,September_Program,September_Donasi,September_Tanggal,September_Day,September_Time,September_Payment,Oktober_Program,Oktober_Donasi,Oktober_Tanggal,Oktober_Day,Oktober_Time,Oktober_Payment,November_Program,November_Donasi,November_Tanggal,November_Day,November_Time,November_Payment,Desember_Program,Desember_Donasi,Desember_Tanggal,Desember_Day,Desember_Time,Desember_Payment,Total,Frekuensi,Preferensi,Program,klasifikasi_program,Label_Jam,Rata - rata,Day_Mode,Date_Category,Date,Bulan_Pertama,Tahun_Pertama,Kategori,Badge,Avg Kategori,Tanggal_terakhir_donasi,Donasi_terakhir,Status,validity,Source,kategori_update,kategori_prioritas,terakhir_interaksi,start_date,kategori_karantina
80061,62812185464542,Nn,NaN,January,2026.0,BERSAMA RINGANKAN DERITA SUMATERA,53549.0,2026-01-01 11:05:56,Thu,11-12,BCA VA,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,53549.0,1.0,1.0,BERSAMA RINGANKAN DERITA SUMATERA,SUMATERA,11-12,53549.0,Thu,Masa Gajian,2026-01-01 11:05:56,1.0,2026.0,Aktif,BRONZE,< 100.000,2026-01-01 11:05:56,53549.0,Pernah Sukses,Number not registered,Web Ads,NaN,NaN,NaT,NaT,NaN
80560,6281296032425,Aisah Merdeka,WA_1092,January,2026.0,PEDULI KORBAN BANJIR SUMATERA,100244.0,2026-01-02 17:13:01,Fri,17-18,GOPAY,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,100244.0,1.0,1.0,PEDULI KORBAN BANJIR SUMATERA,SUMATERA,17-18,100244.0,Fri,Jumat,2026-01-02 17:13:01,1.0,2026.0,Aktif,SILVER,100.000 - 1000.000,2026-01-02 17:13:01,100244.0,Pernah Sukses,Number registered,Web Ads,NaN,NaN,NaT,NaT,NaN
80561,6281315451459,Jean,WA_1092,January,2026.0,PEDULI KORBAN BANJIR SUMATERA,100098.0,2026-01-02 17:20:44,Fri,17-18,SHOPEEPAY,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,100098.0,1.0,1.0,PEDULI KORBAN BANJIR SUMATERA,SUMATERA,17-18,100098.0,Fri,Jumat,2026-01-02 17:20:44,1.0,2026.0,Aktif,SILVER,100.000 - 1000.000,2026-01-02 17:20:44,100098.0,Pernah Sukses,Number registered,Web Ads,NaN,NaN,NaT,NaT,NaN
80562,6287780963045,Hamid,WA_1092,January,2026.0,PEDULI KORBAN BANJIR SUMATERA,100283.0,2026-01-02 17:40:23,Fri,17-18,BRI,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,100283.0,1.0,1.0,PEDULI KORBAN BANJIR SUMATERA,SUMATERA,17-18,100283.0,Fri,Jumat,2026-01-02 17:40:23,1.0,2026.0,Aktif,SILVER,100.000 - 1000.000,2026-01-02 17:40:23,100283.0,Pernah Sukses,Number registered,Web Ads,NaN,NaN,NaT,NaT,NaN
80563,6285234971911,Nur Asiah,WA_1092,January,2026.0,BANTUAN KEMANUSIAAN UNTUK SUDAN,100156.0,2026-01-02 20:44:21,Fri,20-21,OVO,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT

In [129]:
df_summary_database = df_output_final[['Whatsapp',
 'Donatur',
 'Bulan',
 'Tahun',
 'CRM',
 'Source',
 'Total',
 'Frekuensi',
 'klasifikasi_program',
 'Preferensi',
 'Label_Jam',
 'Rata - rata',
 'Day_Mode',
 'Date_Category',
 'Tahun_Pertama',
 'Kategori',
 'Badge',
 'Avg Kategori','Tanggal_terakhir_donasi',"Donasi_terakhir","terakhir_interaksi",'Status', 'validity','kategori_karantina']]

df_summary_database.to_excel("Database.xlsx")

In [130]:
df_summary_database[df_summary_database['CRM'].isna()]['validity'].value_counts(dropna=False)

validity
NaN                      9501
Number registered        8130
Number not registered    6037
Name: count, dtype: int64

In [131]:
# tracking data sukses
data_sukses = df_output_final[df_output_final['Status'] == "Pernah Sukses"]

# tracking data waiting
data_waiting = df_output_final[df_output_final['Status'] == 'Tetap Waiting']

In [132]:
df_summary_database_sukses = data_sukses[['Whatsapp',
 'Donatur',
 'Bulan',
 'Tahun',
 'CRM',
 'Source',
 'Total',
 'Frekuensi',
 'klasifikasi_program',
 'Preferensi',
 'Label_Jam',
 'Rata - rata',
 'Day_Mode',
 'Date_Category',
 'Tahun_Pertama',
 'Kategori',
 'Badge',
 'Avg Kategori','Tanggal_terakhir_donasi',"Donasi_terakhir","terakhir_interaksi",'Status', 'validity','kategori_karantina']]

df_summary_database_waiting = data_waiting[['Whatsapp',
 'Donatur',
 'Bulan',
 'Tahun',
 'CRM',
 'Source',
 'Total',
 'Frekuensi',
 'klasifikasi_program',
 'Preferensi',
 'Label_Jam',
 'Rata - rata',
 'Day_Mode',
 'Date_Category',
 'Tahun_Pertama',
 'Kategori',
 'Badge',
 'Avg Kategori','Tanggal_terakhir_donasi',"Donasi_terakhir","terakhir_interaksi",'Status', 'validity','kategori_karantina']]

df_summary_karantina = df_summary_database[df_summary_database['kategori_karantina'] == "Pasif"]

In [133]:
# database sukses
df_summary_database_sukses.to_excel('Database_sukses.xlsx')

# database waiting
df_summary_database_waiting.to_excel('Database_waiting.xlsx')

# database karantina
df_summary_karantina.to_excel('Database_karantina.xlsx')

In [134]:
data_with_rekapan_updated['Katasdfkljalskdjfri'].value_sadcounts(dropna=False)

KeyError: 'Katasdfkljalskdjfri'

# backup data

In [ ]:
# get data from current backup

username_backup = os.getenv("USERNAME_BU")
password_backup = os.getenv("PASSWORD_BU")
host_backup = os.getenv("HOST_BU")
port_backup = os.getenv("PORT_BU")
database_backup = os.getenv("DATABASE_BU")

# URL-encode username & password in case they have special characters
username_enc = quote_plus(username_backup)
password_enc = quote_plus(password_backup)

# Create SQLAlchemy engine
engine = create_engine(
    f"mysql+pymysql://{username_enc}:{password_enc}@{host_backup}:{port_backup}/{database_backup}"
)

In [ ]:
query_backup = """SELECT *
FROM temanbaik_backup.tb_backup_26_mei_25_july"""

data_backup = pd.read_sql(query_backup, engine)

In [ ]:
# get data from the updated database prod on app
data_updater = donasi_users[['campaign_id','invoice_id', 'name', 'whatsapp', 'email', 'ip', 'comment', 'title','nominal', 'payment_method', 'payment_number', 'payment_account', 'status', 'nominal_commission', 'display_name', 'aff_code', 'created_at_x', 'info_qurban', 'info_package2', 'info_zfitrah', 'info_donate', 'utm_source', 'utm_medium', 'utm_content', 'utm_campaign', 'utm_term', 'utm_id']]

In [ ]:
data_updater = data_updater.dropna(subset=["invoice_id"])
data_backup = data_backup.drop_duplicates(subset=["invoice_id"])

In [ ]:
backup_result = pandas_upsert_with_stats(
    df_main=data_backup,
    df_update=data_updater,
    key="invoice_id"
)

UPSERT SUMMARY
----------------------------------------
Rows updated        : 4680
Rows inserted       : 3459
Columns added       : 0
Column changes:
  nominal         : 26
  email           : 1
  name            : 21
  status          : 71
  comment         : 4596
----------------------------------------


In [ ]:
backup_result[(backup_result['created_at_x']>= "2025-11-01") & (backup_result['created_at_x']<= "2025-12-01")]['status'].value_counts()

status
1.0    7105
0.0    1258
Name: count, dtype: int64

In [ ]:
# start_date = data_backup['created_at_x'].min().strftime('%Y-%m-%d')
# end_date = (pd.Timestamp.today() - pd.DateOffset(months=1)).strftime('%Y-%m-%d')

# data_backup_teman_baik = pd.concat([data_backup_tb, data_backup_tb], ignore_index=True)
backup_result.to_sql("tb_backup_26_mei_25_july", con=engine, if_exists="replace", index=False)

103077